In [4]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

## 단어와 문자의 원-핫 인코딩
* 원-핫 인코딩은 토큰을 벡터로 변환하는 가장 일반적이고 기본적인 방법임
* 모든 단어에 고유한 정수 인덱스를 부여하고 이 정수 인덱스 i를 크기가 N(어휘 사전의 크기)인 이진 벡터로 변환함
* 이 벡터는 i번째 원소만 1이고 나머지는 모두 0임
* 물론 원-핫 인코딩은 문자 수준에서도 적용할 수 있음
* 케라스에는 원본 텍스트 데이터를 단어 또는 문자 수준의 원-핫 인코딩으로 변환해주는 유틸리티가 있음
* 특수 문자를 제거하거나 빈도가 높은 N개의 단어만을 선택(입력 벡터 공간이 너무 커지지 않도록 하기 위한 일반적인 제한 방법입니다)하는 등 여러 가지 중요한 기능들이 있음

케라스를 사용한 단어 수준의 원-핫 인코딩:

In [6]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
# text = 'You say goodbye and I say hello.'

# 가장 빈도가 높은 1,000개의 단어만 선택하도록 Tokenizer 객체를 만듭니다.
tokenizer = Tokenizer(num_words=1000)
# 단어 인덱스를 구축합니다.
tokenizer.fit_on_texts(samples)

# 계산된 단어 인덱스를 구합니다.
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print(word_index)

Found 9 unique tokens.
{'the': 1, 'cat': 2, 'sat': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}


In [7]:
# 문자열을 정수 인덱스의 리스트로 변환합니다.
sequences = tokenizer.texts_to_sequences(samples)
print(sequences)

[[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]


In [8]:
# 직접 원-핫 이진 벡터 표현을 얻을 수 있습니다.
# 원-핫 인코딩 외에 다른 벡터화 방법들도 제공합니다!
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')
print(one_hot_results)

[[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


## 원-핫 인코딩의 변종 중 하나인 원-핫 해싱 기법
* 이 방식은 어휘 사전에 있는 고유한 토큰의 수가 너무 커서 모두 다루기 어려울 때 사용함
* 각 단어에 명시적으로 인덱스를 할당하고 이 인덱스를 딕셔너리에 저장하는 대신에 단어를 해싱하여 고정된 크기의 벡터로 변환함
* 일반적으로 간단한 해싱 함수를 사용함
* 이 방식의 주요 장점은 명시적인 단어 인덱스가 필요 없기 때문에 메모리를 절약하고 온라인 방식으로 데이터를 인코딩할 수 있음(전체 데이터를 확인하지 않고 토큰을 생성할 수 있음)
* 한 가지 단점은 해시 충돌임, 두 개의 단어가 같은 해시를 만들면 이를 바라보는 머신 러닝 모델은 단어 사이의 차이를 인식하지 못하며, 해싱 공간의 차원이 해싱될 고유 토큰의 전체 개수보다 훨씬 크면 해시 충돌의 가능성은 감소함

해싱 기법을 사용한 단어 수준의 원-핫 인코딩(간단한 예):

In [9]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 단어를 크기가 1,000인 벡터로 저장합니다.
# 1,000개(또는 그이상)의 단어가 있다면 해싱 충돌이 늘어나고 인코딩의 정확도가 감소될 것입니다
dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        # 단어를 해싱하여 0과 1,000 사이의 랜덤한 정수 인덱스로 변환합니다.
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.
        print('word(%s) --> i:%d, j:%d, index:%d' % (word, i, j, index))

word(The) --> i:0, j:0, index:139
word(cat) --> i:0, j:1, index:109
word(sat) --> i:0, j:2, index:724
word(on) --> i:0, j:3, index:330
word(the) --> i:0, j:4, index:904
word(mat.) --> i:0, j:5, index:164
word(The) --> i:1, j:0, index:139
word(dog) --> i:1, j:1, index:990
word(ate) --> i:1, j:2, index:102
word(my) --> i:1, j:3, index:998
word(homework.) --> i:1, j:4, index:443
